In [51]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, contains, when, sum, avg, round,max, row_number, desc, collect_list, array_join
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType

In [52]:
spark = SparkSession.builder.appName("ExtractNestedData").getOrCreate()

In [53]:
df = spark.read.csv("/Users/benjaminbrooke/Desktop/Data/dicubed_sdtm_csv_2019-05-30/tr.csv",header = True)
df = df.drop("_c0")

In [54]:
df = df.withColumn("id",row_number().over(Window.partitionBy("STUDYID").orderBy("TRDTC")))

In [55]:
df = df.drop("STUDYID", "DOMAIN","TRORRES","TRORRESU","TRSTRESC")

In [56]:
df.show(10)

+----------+-----+-------+--------+----------------+--------+--------+--------+--------+-----+----------+---+
|   USUBJID|TRSEQ|TRLNKID|TRTESTCD|          TRTEST|TRSTRESN|TRSTRESU|TRMETHOD|VISITNUM|VISIT|     TRDTC| id|
+----------+-----+-------+--------+----------------+--------+--------+--------+--------+-----+----------+---+
|UCSF-BR-09|   41|    T01|  VOLUME|          Volume|    29.9|      mL|     MRI|       1|  MR1|1986-11-26|  1|
|UCSF-BR-09|   42|    T01|   LDIAM|Longest Diameter|      64|      mm|     MRI|       1|  MR1|1986-11-26|  2|
|UCSF-BR-09|   43|    T01|  VOLUME|          Volume|      20|      mL|     MRI|       3|  MR3|1987-01-27|  3|
|UCSF-BR-09|   44|    T01|   LDIAM|Longest Diameter|      64|      mm|     MRI|       3|  MR3|1987-01-27|  4|
|UCSF-BR-67|  369|    T01|   LDIAM|Longest Diameter|    61.7|      mm|     MRI|       1|  MR1|1988-04-21|  5|
|UCSF-BR-67|  370|    T01|  VOLUME|          Volume|    30.1|      mL|     MRI|       1|  MR1|1988-04-21|  6|
|UCSF-BR-1

In [57]:
df = df.withColumn("collect", when(col("TRSTRESU")=="mL",collect_list("TRSEQ").over(Window.partitionBy("USUBJID")))
                   .otherwise(when(col("TRSTRESU")=="mm",collect_list("TRSEQ").over(Window.partitionBy("USUBJID")))))

In [58]:
df.show(20)

+----------+-----+-------+--------+----------------+--------+--------+--------+--------+-----+----------+---+----------------+
|   USUBJID|TRSEQ|TRLNKID|TRTESTCD|          TRTEST|TRSTRESN|TRSTRESU|TRMETHOD|VISITNUM|VISIT|     TRDTC| id|         collect|
+----------+-----+-------+--------+----------------+--------+--------+--------+--------+-----+----------+---+----------------+
|ISPY1_1001|    1|    T01|   LDIAM|Longest Diameter|      88|      mm|     MRI|       1|  MR1|1984-10-13|  1|    [1, 2, 3, 4]|
|ISPY1_1001|    2|    T01|   LDIAM|Longest Diameter|      78|      mm|     MRI|       2|  MR2|1984-11-08|  3|    [1, 2, 3, 4]|
|ISPY1_1001|    3|    T01|   LDIAM|Longest Diameter|      30|      mm|     MRI|       3|  MR3|1985-01-10| 10|    [1, 2, 3, 4]|
|ISPY1_1001|    4|    T01|   LDIAM|Longest Diameter|      14|      mm|     MRI|       4|  MR4|1985-04-13| 27|    [1, 2, 3, 4]|
|ISPY1_1002|    5|    T01|   LDIAM|Longest Diameter|      29|      mm|     MRI|       1|  MR1|1984-11-02|  2|  

In [59]:
df = df.withColumn("collect", array_join(col("collect")," | "))

In [66]:
df = df.withColumn("collect_dates", when(col("TRSTRESU")=="mL",collect_list("TRDTC").over(Window.partitionBy("USUBJID")))
                   .otherwise(when(col("TRSTRESU")=="mm",collect_list("TRDTC").over(Window.partitionBy("USUBJID")))))

In [67]:
df = df.withColumn("collect_numbers", when(col("TRSTRESU")=="mL",collect_list("TRSTRESN").over(Window.partitionBy("USUBJID")))
                   .otherwise(when(col("TRSTRESU")=="mm",collect_list("TRSTRESN").over(Window.partitionBy("USUBJID")))))

In [68]:
df.orderBy("USUBJID","TRDTC").show()

+----------+-----+-------+--------+----------------+--------+--------+--------+--------+-----+----------+---+-----------------+--------------------+-----------------+
|   USUBJID|TRSEQ|TRLNKID|TRTESTCD|          TRTEST|TRSTRESN|TRSTRESU|TRMETHOD|VISITNUM|VISIT|     TRDTC| id|          collect|       collect_dates|  collect_numbers|
+----------+-----+-------+--------+----------------+--------+--------+--------+--------+-----+----------+---+-----------------+--------------------+-----------------+
|ISPY1_1001|    1|    T01|   LDIAM|Longest Diameter|      88|      mm|     MRI|       1|  MR1|1984-10-13|  1|    1 | 2 | 3 | 4|[1984-10-13, 1984...| [88, 78, 30, 14]|
|ISPY1_1001|    2|    T01|   LDIAM|Longest Diameter|      78|      mm|     MRI|       2|  MR2|1984-11-08|  3|    1 | 2 | 3 | 4|[1984-10-13, 1984...| [88, 78, 30, 14]|
|ISPY1_1001|    3|    T01|   LDIAM|Longest Diameter|      30|      mm|     MRI|       3|  MR3|1985-01-10| 10|    1 | 2 | 3 | 4|[1984-10-13, 1984...| [88, 78, 30, 14]

In [69]:
df = df.drop("TRLNKID","TRDTC")

In [70]:
df.show()

+----------+-----+--------+----------------+--------+--------+--------+--------+-----+---+-----------------+--------------------+-----------------+
|   USUBJID|TRSEQ|TRTESTCD|          TRTEST|TRSTRESN|TRSTRESU|TRMETHOD|VISITNUM|VISIT| id|          collect|       collect_dates|  collect_numbers|
+----------+-----+--------+----------------+--------+--------+--------+--------+-----+---+-----------------+--------------------+-----------------+
|ISPY1_1001|    1|   LDIAM|Longest Diameter|      88|      mm|     MRI|       1|  MR1|  1|    1 | 2 | 3 | 4|[1984-10-13, 1984...| [88, 78, 30, 14]|
|ISPY1_1001|    2|   LDIAM|Longest Diameter|      78|      mm|     MRI|       2|  MR2|  3|    1 | 2 | 3 | 4|[1984-10-13, 1984...| [88, 78, 30, 14]|
|ISPY1_1001|    3|   LDIAM|Longest Diameter|      30|      mm|     MRI|       3|  MR3| 10|    1 | 2 | 3 | 4|[1984-10-13, 1984...| [88, 78, 30, 14]|
|ISPY1_1001|    4|   LDIAM|Longest Diameter|      14|      mm|     MRI|       4|  MR4| 27|    1 | 2 | 3 | 4|[198